In [1]:
from keras.layers import Input , Dense , LSTM , Attention , Embedding , dot ,concatenate
from keras.models import  Model
from keras.preprocessing.text import  Tokenizer
from keras.preprocessing.sequence import  pad_sequences

2023-12-08 13:55:04.482856: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-08 13:55:04.541250: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-08 13:55:04.542257: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 13:55:05.675159: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
french_sentences = ['Bonjour', 'Comment ça va?', 'La vie est belle']
english_sentences = ['Hello', 'How are you?', 'Life is beautiful']

In [3]:
french_tokenizer = Tokenizer(filters='')
english_tokenizer = Tokenizer(filters='')

french_tokenizer.fit_on_texts(french_sentences)
english_tokenizer.fit_on_texts(english_sentences)

In [4]:
french_tokenizer.index_word

{1: 'bonjour',
 2: 'comment',
 3: 'ça',
 4: 'va?',
 5: 'la',
 6: 'vie',
 7: 'est',
 8: 'belle'}

In [5]:
english_tokenizer.index_word

{1: 'hello', 2: 'how', 3: 'are', 4: 'you?', 5: 'life', 6: 'is', 7: 'beautiful'}

In [6]:
max_french_len = max([len(sentence.split()) for sentence in french_sentences])
max_english_len = max([len(sentence.split()) for sentence in english_sentences])

In [7]:
X_french = french_tokenizer.texts_to_sequences(french_sentences)
X_french = pad_sequences( X_french , maxlen=max_french_len , padding="post")

Y_english = english_tokenizer.texts_to_sequences(english_sentences)
Y_english = pad_sequences( Y_english , maxlen=max_english_len , padding="post")

In [8]:
X_french ,Y_english

(array([[1, 0, 0, 0],
        [2, 3, 4, 0],
        [5, 6, 7, 8]], dtype=int32),
 array([[1, 0, 0],
        [2, 3, 4],
        [5, 6, 7]], dtype=int32))

In [9]:
len_french_word_corpus = len(french_tokenizer.word_index) + 1
len_english_word_corpus = len(english_tokenizer.word_index) + 1

In [14]:
latent_dim = 64

encoder_inputs = Input(shape=(max_french_len,))
encoder_embedding = Embedding(input_dim=len_french_word_corpus ,output_dim=latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim ,return_sequences=True ,input_length=encoder_embedding)

decoder_inputs = Input(shape=(max_english_len,))
decoder_embedding = Embedding(input_dim=len_english_word_corpus ,output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim ,return_sequences=True ,input_length=decoder_embedding) 

In [16]:
attention = Attention()([encoder_lstm, decoder_lstm])
context = dot([attention, encoder_lstm], axes=[2, 1])
decoder_combined_context = concatenate([context, decoder_lstm])

output = Dense(len_english_word_corpus ,activation="softmax")(decoder_combined_context)

model = Model(
    inputs=[encoder_inputs,decoder_inputs],
    outputs=output
)

ValueError: Exception encountered when calling layer 'attention_4' (type Attention).

Attempt to convert a value (<keras.src.layers.rnn.lstm.LSTM object at 0x7fea710559d0>) with an unsupported type (<class 'keras.src.layers.rnn.lstm.LSTM'>) to a Tensor.

Call arguments received by layer 'attention_4' (type Attention):
  • inputs=['<keras.src.layers.rnn.lstm.LSTM object at 0x7fea710559d0>', '<keras.src.layers.rnn.lstm.LSTM object at 0x7fea710eac90>']
  • mask=None
  • training=None
  • return_attention_scores=False
  • use_causal_mask=False